<a href="https://colab.research.google.com/github/sgonzalezgg/sd-finetuning-comparison/blob/main/evaluation/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Evaluación**

Script de evaluación del fine-tuning del modelo Stable Diffusion mediante Textual Inversion, LoRA y DreamBooth.

Se hace una evaluación manual en base a 3 criterios: fidelidad, robustez, adaptabilidad estilística y creatividad.

Además se calcula el CLIP Score para analizar la fidelidad texto-imagen

## Configuración inicial

In [ ]:
# !pip install torchmetrics

In [ ]:
!wget https://raw.githubusercontent.com/sgonzalezgg/sd-finetuning-comparison/refs/heads/main/evaluation/eval_tools.py
!wget https://raw.githubusercontent.com/sgonzalezgg/sd-finetuning-comparison/refs/heads/main/evaluation/config.yaml

--2025-06-14 13:45:49--  https://raw.githubusercontent.com/sgonzalezgg/sd-finetuning-comparison/refs/heads/main/evaluation/eval_tools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3552 (3.5K) [text/plain]
Saving to: ‘eval_tools.py’

eval_tools.py       100%[===================>]   3.47K  --.-KB/s    in 0s      

2025-06-14 13:45:50 (54.1 MB/s) - ‘eval_tools.py’ saved [3552/3552]

--2025-06-14 13:45:50--  https://raw.githubusercontent.com/sgonzalezgg/sd-finetuning-comparison/refs/heads/main/evaluation/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, a

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import pandas as pd
import yaml

In [ ]:
from eval_tools import *

## Carga el fichero de configuración

In [ ]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

prompts_fidelidad = config["fidelidad"]["prompts"]
negative_prompts_fidelidad = config["fidelidad"]["negative_prompts"]
names_export_fidelidad = config["fidelidad"]["names_export"]

prompts_robustez = config["robustez"]["prompts"]
negative_prompts_robustez = config["robustez"]["negative_prompts"]
names_export_robustez = config["robustez"]["names_export"]

prompts_estilos = config["estilos"]["prompts"]
negative_prompts_estilos= config["estilos"]["negative_prompts"]
names_export_estilos = config["estilos"]["names_export"]

prompts_creatividad = config["creatividad"]["prompts"]
negative_prompts_creatividad = config["creatividad"]["negative_prompts"]
names_export_creatividad = config["creatividad"]["names_export"]

## Carga de los modelos entrenados

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#@title Pipeline para Textual Inversion
pipe_ti = StableDiffusionPipeline.from_pretrained("sgonzalezygil/sd-finetuning-comparison-ti-v2",
                                                  torch_dtype=torch.float16)
pipe_ti.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "sgonzalezygil/sd-finetuning-comparison-ti-v2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
#@title Pipeline para LoRA
pipe_lora = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5",
                                                    torch_dtype=torch.float16).to("cuda")

# Cargar LoRA usando compatibilidad con Diffusers:
pipe_lora.load_lora_weights("sgonzalezygil/sd-finetuning-lora-v2",
                            weight_name="zapatillasLoRA-06.safetensors")
pipe_lora.fuse_lora()

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

zapatillasLoRA-06.safetensors:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

In [ ]:
#@title Pipeline para Dreambooth
pipe_db = StableDiffusionPipeline.from_pretrained("sgonzalezygil/sd-finetuning-dreambooth",
                                                  torch_dtype=torch.float16)
pipe_db.to("cuda")

model_index.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "sgonzalezygil/sd-finetuning-dreambooth",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

## Inicio de la generación y evaluación

In [ ]:
# Parámetros base
# num_samples = 3
# num_rows = 1
# height = 512
# width = 512
# guidance_scale = 7.5
# num_inference_steps = 50

### **Fidelidad**

In [ ]:
clip_fidelidad_ti = generate_images(
    prompts=prompts_fidelidad,
    names_export=[i + "_TI" for i in names_export_fidelidad],
    negative_prompts=negative_prompts_fidelidad,
    pipe=pipe_ti
    )

print(clip_fidelidad_ti)

In [ ]:
clip_fidelidad_lora = generate_images(
    prompts=prompts_fidelidad,
    names_export=[i + "_LoRA" for i in names_export_fidelidad],
    negative_prompts=negative_prompts_fidelidad,
    pipe=pipe_lora
    )

print(clip_fidelidad_lora)

In [ ]:
clip_fidelidad_db = generate_images(
    prompts=prompts_fidelidad,
    names_export=[i + "_DB" for i in names_export_fidelidad],
    negative_prompts=negative_prompts_fidelidad,
    pipe=pipe_db
    )

print(clip_fidelidad_db)

### **Robustez**

In [ ]:
clip_robustez_ti = generate_images(
    prompts=prompts_robustez,
    names_export=[i + "_TI" for i in names_export_robustez],
    negative_prompts=negative_prompts_robustez,
    pipe=pipe_ti
    )

print(clip_robustez_ti)

In [ ]:
clip_robustez_lora = generate_images(
    prompts=prompts_robustez,
    names_export=[i + "_LoRA" for i in names_export_robustez],
    negative_prompts=negative_prompts_robustez,
    pipe=pipe_lora
    )

print(clip_robustez_lora)

In [ ]:
clip_robustez_db = generate_images(
    prompts=prompts_robustez,
    names_export=[i + "_DB" for i in names_export_robustez],
    negative_prompts=negative_prompts_robustez,
    pipe=pipe_db
    )

print(clip_robustez_db)

### **Adaptabilidad estilística**

In [ ]:
clip_estilos_ti = generate_images(
    prompts=prompts_estilos,
    names_export=[i + "_TI" for i in names_export_estilos],
    negative_prompts=negative_prompts_estilos,
    pipe=pipe_ti
    )

print(clip_estilos_ti)

In [ ]:
clip_estilos_lora = generate_images(
    prompts=prompts_estilos,
    names_export=[i + "_LoRA" for i in names_export_estilos],
    negative_prompts=negative_prompts_estilos,
    pipe=pipe_lora
    )

print(clip_estilos_lora)

In [ ]:
clip_estilos_db = generate_images(
    prompts=prompts_estilos,
    names_export=[i + "_DB" for i in names_export_estilos],
    negative_prompts=negative_prompts_estilos,
    pipe=pipe_db
    )

print(clip_estilos_db)

### **Creatividad**

In [ ]:
clip_creatividad_ti = generate_images(
    prompts=prompts_creatividad,
    names_export=[i + "_TI" for i in names_export_creatividad],
    negative_prompts=negative_prompts_creatividad,
    pipe=pipe_ti
    )

print(clip_creatividad_ti)

In [ ]:
clip_creatividad_lora = generate_images(
    prompts=prompts_creatividad,
    names_export=[i + "_LoRA" for i in names_export_creatividad],
    negative_prompts=negative_prompts_creatividad,
    pipe=pipe_lora
    )

print(clip_creatividad_lora)

In [ ]:
clip_creatividad_db = generate_images(
    prompts=prompts_creatividad,
    names_export=[i + "_DB" for i in names_export_creatividad],
    negative_prompts=negative_prompts_creatividad,
    pipe=pipe_db
    )

print(clip_creatividad_db)

## Tablas resumen

In [ ]:
import pandas as pd
fidelidad = [clip_fidelidad_ti[0], clip_fidelidad_lora[0], clip_fidelidad_db[0]]
robustez = [clip_robustez_ti[0], clip_robustez_lora[0], clip_robustez_db[0]]
estilos = [clip_estilos_ti[0], clip_estilos_lora[0], clip_estilos_db[0]]
creatividad = [clip_creatividad_ti[0], clip_creatividad_lora[0], clip_creatividad_db[0]]
df = pd.DataFrame({
    "Fidelidad": fidelidad,
    "Robustez": robustez,
    "Adaptabilidad estilística": estilos,
    "Creatividad": creatividad

})

df["Promedio"] = df.mean(axis=1)
df.index=["Textual Inversion", "LoRA", "DreamBooth"]

In [ ]:
df

,Fidelidad,Robustez,Adaptabilidad estilística,Creatividad,Promedio
Textual Inversion,27.191101,29.2284,26.184328,31.092207,28.424009
LoRA,27.504082,29.626661,29.159054,30.429108,29.179726
DreamBooth,27.790731,28.03562,28.845299,30.174582,28.711557


In [ ]:
df_all = pd.DataFrame({
    "Textual inversion": clip_fidelidad_ti[1] + clip_robustez_ti[1] + clip_estilos_ti[1] + clip_creatividad_ti[1],
    "LoRA": clip_fidelidad_lora[1] + clip_robustez_lora[1] + clip_estilos_lora[1] + clip_creatividad_lora[1],
    "DreamBooth": clip_fidelidad_db[1] + clip_robustez_db[1] + clip_estilos_db[1] + clip_creatividad_db[1]
})

prompts_fidelidad_clean = [' '.join(s.split()) for s in prompts_fidelidad]
prompts_robustez_clean = [' '.join(s.split()) for s in prompts_robustez]
prompts_estilos_clean = [' '.join(s.split()) for s in prompts_estilos]
prompts_creatividad_clean = [' '.join(s.split()) for s in prompts_creatividad]

list_prompts = prompts_fidelidad_clean + prompts_robustez_clean + prompts_estilos_clean + prompts_creatividad_clean

df_all.index=list_prompts

In [ ]:
df_all

,Textual inversion,LoRA,DreamBooth
"A pair of <zptx> sneakers, white background, photorealistic, high resolution, studio lighting",27.189966,29.632133,29.949707
"Close-up of the <zptx> sneaker. Side view, fabric texture visible, macro shot",28.887014,27.408030,28.193993
"A clean minimalist ad featuring the blue and white <zptx> sneakers. Solid color background. High contrast, product focus, no text, modern design magazine style. Realistic",21.985008,22.873255,23.257010
"A digital eco-themed ad showing the <zptx> sneakers resting on a moss-covered rock in a futuristic forest, with soft morning light filtering through trees. High detail, green and silver tones, peaceful yet modern aesthetic",29.016939,27.329590,27.089058
"A digital ad showing the blue and white <zptx> sneaker resting on a beach. Sunset light, high detail, warm tones, peaceful yet modern aesthetic",28.876593,30.277395,30.463890
A red and white <zptx> running sneaker,28.085546,27.515770,25.235405
"Detailed photo of a single <zptx> sneaker on a light yellow acrylic platform. Studio lighting with soft shadows, realistic texture, professional commercial aesthetic",28.003531,29.932724,27.805490
"Professional studio photo of lots of <zptx> running sneakers arranged in a circle, soft shadows, realistic, color-blocking composition",29.741945,30.305389,29.443405
"A top-down view of a <zptx> training sneaker placed on lush green grass, realistic lighting, soft shadows, high resolution",31.082560,30.752756,29.658159
"Futuristic <zptx> sneaker glowing with neon pink and cyan circuit patterns,rain-soaked Tokyo street at night, cyberpunk aesthetic, cinematic lighting",24.760805,28.553909,26.711287


In [ ]:
import openpyxl

In [ ]:
df.to_excel("CLIP_por_criterio.xlsx", index=True)

In [ ]:
df_all.to_excel("CLIP_por_prompt.xlsx", index=True)

In [ ]:
!zip -r generated_images.zip generated_images

  adding: generated_images/ (stored 0%)
  adding: generated_images/C1_LoRA.png (deflated 0%)
  adding: generated_images/C6_LoRA.png (deflated 0%)
  adding: generated_images/E3_LoRA.png (deflated 0%)
  adding: generated_images/C3_LoRA.png (deflated 0%)
  adding: generated_images/C5_DB.png (deflated 0%)
  adding: generated_images/E2_DB.png (deflated 0%)
  adding: generated_images/F4_LoRA.png (deflated 0%)
  adding: generated_images/R4_LoRA.png (deflated 0%)
  adding: generated_images/F2_DB.png (deflated 0%)
  adding: generated_images/R4_TI.png (deflated 0%)
  adding: generated_images/C2_LoRA.png (deflated 0%)
  adding: generated_images/F3_DB.png (deflated 0%)
  adding: generated_images/E4_LoRA.png (deflated 0%)
  adding: generated_images/R4_DB.png (deflated 0%)
  adding: generated_images/C2_DB.png (deflated 0%)
  adding: generated_images/E3_DB.png (deflated 0%)
  adding: generated_images/C4_DB.png (deflated 0%)
  adding: generated_images/C5_LoRA.png (deflated 0%)
  adding: generated_imag

In [ ]:
from google.colab import files
files.download('generated_images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>